In [24]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit,transpile,Aer,execute,ClassicalRegister,QuantumRegister,assemble,execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os
import hashlib
from random import randint,choice
import numpy as np

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [25]:
q=5
# create an Alice qubit register
alice_qubits=QuantumRegister(q)

# create a classical bit register for the key exchange
key_exchange=ClassicalRegister(q)

# create a Quantum Circuit with the registers
circuit=QuantumCircuit(alice_qubits,key_exchange)

# generate random bit string for Alice to encode
alice_bits=[randint(0,1) for _ in range(q)]
alice_choices=[choice(['NS','S']) for _ in range(q)]

# encode Alice's bits into qubits
for i,bit in enumerate(alice_bits):
    if bit==1:
        circuit.x(alice_qubits[i])
    else:
        circuit.z(alice_qubits[i])

    if alice_choices[i]=='S':
        circuit.h(alice_qubits[i]) 

circuit.barrier()

In [26]:
# Alice and Bob share their qubits
#circuit+=alice_qubits+bob_qubits

# generate random bit string for Bob to measure the qubits
#bob_bits=[randint(0,1) for _ in range(q)]
bob_choices=[choice(['NS','S']) for _ in range(q)]
    
# measure the qubits
for i in range(len(bob_choices)):
    if bob_choices[i]=='S':
        circuit.h(i)

    circuit.measure(i,i)

circuit.barrier()

In [27]:
bob_bits=[]

backend=Aer.get_backend('qasm_simulator')
job=execute(circuit,backend=backend,shots=1,memory=True)
result=job.result()
counts=result.get_counts()
result=list(counts.keys())[0]
for i in range(len(result)):
    bob_bits.append(int(result[i]))
print(bob_bits)

[0, 0, 1, 0, 0]


In [29]:
# Alice and Bob compare their bit strings to identify the positions where they used the same basis
# and therefore can share a secure key
# matching_bits=[i for i in range(q) if alice_choices[i]==bob_choices[i]]
# alice_key=[alice_bits[i] for i in matching_bits]
# bob_key=[bob_bits[i] for i in matching_bits]

alice_key=[]
for q in range(len(alice_choices)):
    if alice_choices[q]==bob_choices[q]:
        alice_key.append(alice_bits[q])

bob_key=[]
for q in range(len(bob_choices)):
    if alice_choices[q]==bob_choices[q]:
        bob_key.append(bob_bits[q])

print(alice_key)
print(bob_key)

t=0
if len(alice_key) and len(bob_key):
    for i in range(len(alice_key)):
        if alice_key[i]!=bob_key[i]:
            t+=1

# check if enough matching bits were found to generate the key
if t<1:
    print("Users verified")
    # generate the key from the matching bits
    key_string=''.join(str(bit) for bit in alice_key)
    key_bytes=bytes(int(key_string[i:i+8],2) for i in range(0,len(key_string),8))
    # if len(key_bytes)>=16:
    qkd_key=key_bytes[:16]
    # encrypt a message using AES
    message=input("Enter message: ")
    hash_object=hashlib.sha256(message.encode('utf-8'))
    hex_dig=hash_object.hexdigest()
    hash_bytes=bytes.fromhex(hex_dig)
    message=message.encode('utf-8')  
    cipher=AES.new(hash_bytes,AES.MODE_CBC)
    ciphertext=cipher.encrypt(pad(message,AES.block_size))

    print("Confidentiality-\nCiphertext:",ciphertext)
    print("QKD Key:",qkd_key)

    # decrypt the message using AES-256
    decipher=AES.new(hash_bytes,AES.MODE_CBC,iv=cipher.iv)
    plaintext=str(unpad(decipher.decrypt(ciphertext),AES.block_size))
    hash_obj=hashlib.sha256(plaintext.encode('utf-8'))
    if hash_bytes==bytes.fromhex(hash_obj.hexdigest()):
        print("message authenticated")

    print("Plaintext:",plaintext)
    # else:
    #     print("Not enough bits to generate the key, Let's use another quantum channel.")
else:
    print("EVE DETECTED! or Not enough matching bits to generate the key, Let's use another quantum channel.")

[0, 0, 1, 0]
[0, 0, 1, 0]
Users verified


Enter message:  I can Win CodXpress


Confidentiality-
Ciphertext: b';R\xdc\x03\x1aSp}\xe4"\x0f)\xf0Z\x93\xa7|\xb3Z:\xe9\xe7\x19\x9c\x88\x91\xae\x01\xa7\x1c\x85\xda'
QKD Key: b'\x02'
Plaintext: b'I can Win CodXpress'
